In [1]:
from main_2 import generate_script_gemini

c:\Users\GLB-BLR-307\Documents\VideoGeneration\venv_vid_gen_312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
secs = 50
secs_for_each_clip = 10
number_of_clips_to_merge = secs // secs_for_each_clip

In [6]:
number_of_clips_to_merge

5

In [7]:
gemini_response, raw = generate_script_gemini(instruction="Create an add for employee monitoring system and its ease of integration with the workspace", brand_name="EMP Monitor", tag_line="Time matters", sale=True,time_spans=[str(secs_for_each_clip)+" secs" for i in range(number_of_clips_to_merge)])


Based on the instruction prepare a video ad script that will be displayed over the video as subtitles.
                                      Since I have merged few videos of different time span to make this video, Consider the script to be divided into the repective time-spans in the list:
                                      ['10 secs', '10 secs', '10 secs', '10 secs', '10 secs'],
                                      
                                      Prepare a script for INSTRUCTION:"General purpose Create an add for employee monitoring system and its ease of integration with the workspace".
                                                           Other_information: Brand name is "Emp monitor", Tag line of brand is "Time matters", Our sale is also live.
                                      
                                      Pattern: [<str>,<str>] make one short and another a longer sentence for each.
                                      HINT: Based on this generated te

In [8]:
gemini_response

['EmpMonitor: Time Matters', 'Effortless employee monitoring.']